In [6]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained('xlm-roberta-base')
len(tok)

250002

In [4]:
tok = AutoTokenizer.from_pretrained('Shitao/RetroMAE_MSMARCO')
len(tok)

30522

In [1]:
import os
import json
chunk_names = [fname for fname in os.listdir('data/augmented/back-translate/') if fname.endswith('.json')]
chunk_names[414]

'500.json'

## pyarrow dataset

In [ ]:
batch = [
    {
        "content": [
            {
                "speaker": 0,
                "utterance": "Hey."
            },
            {
                "speaker": 1,
                "utterance": "Hey, what can I do for you?"
            },
            {
                "speaker": 0,
                "utterance": "I'd like to visit Chicago, Il-Ord, to meet my best friend, so I need to book a plane ticket."
            },
            {
                "speaker": 1,
                "utterance": "Okay."
            },
            {
                "speaker": 0,
                "utterance": "Could you book me a ticket for 09/09?"
            },
            {
                "speaker": 1,
                "utterance": "Please send me the place of origin."
            },
            {
                "speaker": 0,
                "utterance": "My name is Stephanie Carter."
            },
            {
                "speaker": 1,
                "utterance": "May I ask the date of arrival, please?"
            },
            {
                "speaker": 0,
                "utterance": "I'd like to arrive at 09/11."
            },
            {
                "speaker": 1,
                "utterance": "Please send the exact location of your origin."
            },
            {
                "speaker": 0,
                "utterance": "Of course, my origins are Dallas and Fort Worth, TX - DFW."
            },
            {
                "speaker": 1,
                "utterance": "Do you have any other specifications?"
            },
            {
                "speaker": 0,
                "utterance": "Yeah, I need to stop between my journey."
            },
            {
                "speaker": 1,
                "utterance": "I found a Hawaiian airline number 1003, which meets your criteria."
            },
            {
                "speaker": 0,
                "utterance": "All right, please book the tickets."
            },
            {
                "speaker": 1,
                "utterance": "Your ticket is booked."
            },
            {
                "speaker": 0,
                "utterance": "Thank you."
            },
            {
                "speaker": 1,
                "utterance": "Thank you."
            }
        ],
        "source_dataset_name": "AirDialogue",
        "idx_within_source": 187590,
        "idx": 0
    },
    {
        "content": [
            {
                "speaker": 0,
                "utterance": "Hey."
            },
            {
                "speaker": 1,
                "utterance": "Hey, what can I do for you today?"
            },
            {
                "speaker": 0,
                "utterance": "I'm Anthony Martinez from Charlotte, I want to go to Orlando this month to attend the summit, could you book a ticket from the CLT to MCO?"
            },
            {
                "speaker": 1,
                "utterance": "I'm here to help you with that."
            },
            {
                "speaker": 0,
                "utterance": "Thank you, I want to leave for 08/13 and book my return ticket for 08/15, too."
            },
            {
                "speaker": 1,
                "utterance": "Wait a minute."
            },
            {
                "speaker": 0,
                "utterance": "Okay."
            },
            {
                "speaker": 1,
                "utterance": "There's a one-stop flight with a 100 ticket rate, which is provided by Jet Blue airlines."
            },
            {
                "speaker": 0,
                "utterance": "Please continue."
            },
            {
                "speaker": 1,
                "utterance": "I booked your flight 1026."
            },
            {
                "speaker": 0,
                "utterance": "Thank you for your help."
            },
            {
                "speaker": 1,
                "utterance": "Greetings, thank you for choosing our service."
            }
        ],
        "source_dataset_name": "AirDialogue",
        "idx_within_source": 285622,
        "idx": 1
    }
]

In [40]:
names = [
    'MS-DC',
    'MetaLWOZ',
    'MULTIWOZ2_2',
    'SGD',
    'SimJointGEN',
    'KETOD',
    'FRAMES',
    'Disambiguation',
    'ABCD',
    'AirDialogue',
    'BiTOD',
    'Taskmaster1'
]

upper_bound = 96

upper_bounds = {
    'MS-DC': min(upper_bound, 250),
    'MetaLWOZ': min(upper_bound, 100),
    'MULTIWOZ2_2': min(upper_bound, 75),
    'SGD': None,
    'SimJointGEN': upper_bound,
    'KETOD': upper_bound,
    'FRAMES': upper_bound,
    'Disambiguation': min(upper_bound, 60),
    'ABCD': upper_bound,
    'AirDialogue': upper_bound,
    'BiTOD': upper_bound,
    'Taskmaster1': min(upper_bound, 200),
}


from random import shuffle, seed as set_seet
from math import ceil
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from functools import partial
from typing import Tuple, List
from tqdm import tqdm
from mylib.utils.data import Dialogue, ContextResponsePair
import json
import os
import pyarrow as pa
from datasets import load_dataset


def preprocess_dialogue(
        raw_sample,
        tokenizer,
        bound=None,
        user_id=0,
        system_id=1,
    ):
    """convert single dialogue (in DialogStudio format) to list of utterances and list of corresponding speaker ids"""
    
    if is_empty(raw_sample) or is_too_long(raw_sample) or has_only_single_utterance(raw_sample):
        return

    utterances = []
    speakers = []
    
    for turn in raw_sample:
        for sp, item in zip([user_id, system_id], ['user utterance', 'system response']):
            ut = turn[item]
            if ut == '':
                continue
            utterances.append(ut)
            speakers.append(sp)
    
    if bound is not None and any(is_above_bound(ut, tokenizer, bound) for ut in utterances):
        # if there're any utterances with exceeding length
        return
    
    return utterances, speakers


def is_empty(dia):
    return len(dia) == 0


def is_too_long(dia):
    return len(dia) > 10


def has_only_single_utterance(dia):
    return len(dia) == 1 and (dia[0]['user utterance'] == '' or dia[0]['system response'] == '')


def is_above_bound(ut, tokenizer, bound):
    return len(tokenizer(ut)['input_ids']) > bound


def get_record_iterator(name, tokenizer, bound):
    dataset = load_dataset('Salesforce/dialogstudio', name)['train']['log']
    for i, raw_dia in enumerate(dataset):
        dia = preprocess_dialogue(raw_dia, tokenizer, bound)
        if dia is None:
            continue
        utterances, speakers = dia
        dia = Dialogue(
            utterances=utterances,
            speakers=speakers,
            source_dataset_name=name,
            idx_within_source=i,
            # idx=None
        )
        dct = dia.asdict()
        dct['content'] = json.dumps(dct['content'])
        yield pa.RecordBatch.from_pylist([dct])


def train_test_split(data: List[Dialogue], frac=0.9, seed=0):
    """resulting sizes:
    - train: `frac`
    - test: `(1 - frac) // 2`
    - val: `(1 - frac) // 2`"""
    
    set_seet(seed)
    shuffle(data)

    # assign indices after shuffling the dataset
    for i, dia in enumerate(data):
        dia.idx = i

    n_total = len(data)
    train_size = ceil(frac * n_total)
    test_size = (n_total - train_size) // 2
    val_size = n_total - train_size - test_size

    res = {
        'train': data[:train_size],
        'test': data[train_size:train_size+test_size],
        'val': data[train_size+test_size:]
    }

    print('dataset splits sizes:')
    print(f'{n_total=}, {train_size=}, {test_size=}, {val_size=}')

    return res


def make_pairs(dialogues):
    res = []
    for dia in tqdm(dialogues, desc='making pairs'):
        pairs = []
        for i in range(len(dia)-1):
            pairs.append((dia[:i+1], dia[i+1]))
        res.extend(pairs)
    shuffle(res)
    res = [ContextResponsePair(context=c, response=r, idx=i) for i, (c, r) in enumerate(res)]
    return res

In [41]:
from transformers import AutoTokenizer
from collections import defaultdict
from mylib.utils.training import seed_everything
import itertools as it
import pyarrow.dataset as ds

seed_everything(0)

# supress warnings about long sequences
import logging
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

#! not the same as roberta, replace in future
tokenizer = AutoTokenizer.from_pretrained('microsoft/mpnet-base')

# load datasets from hugging face, parse, filter and merge into single list
record_iterator_list = []
for dataset_name in names:
    iterator = get_record_iterator(dataset_name, tokenizer, upper_bounds[dataset_name])
    record_iterator_list.append(iterator)

chained_iterator = it.chain.from_iterable(record_iterator_list)

In [42]:
itr = iter(chained_iterator)

In [43]:
next(itr)

pyarrow.RecordBatch
content: string
source_dataset_name: string
idx_within_source: int64
----
content: ["[{"utterance": "I'd like 2 tickets to see Zoolander 2 tomorrow at Regal Meridian 16 theater in Seattle at 9:25 PM", "speaker": 0}, {"utterance": "Okay, your purchase of 2 tickets for Zoolander 2 is confirmed.", "speaker": 1}]"]
source_dataset_name: ["MS-DC"]
idx_within_source: [0]

In [1]:
import pyarrow.dataset as ds

source_dataset = ds.dataset('data-2/source')

In [50]:
source_dataset.take([100500, 100501]).to_pandas()

,content,idx_within_source,source_dataset_name
0,"[{""utterance"": ""Hi, how can I help you?"", ""spe...",81664,SimJointGEN
1,"[{""utterance"": ""Hi, how can I help you?"", ""spe...",81665,SimJointGEN


In [2]:
source_dataset.shuffle()

AttributeError: 'pyarrow._dataset.FileSystemDataset' object has no attribute 'shuffle'